<a href="https://colab.research.google.com/github/OtienoSharon15/multimodal_bot/blob/main/barcelona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio requests pillow #for building an intractive UI interface, making http requests, for image processing

In [ ]:
# imports
import gradio as gr
import requests
import base64
from PIL import Image
import io
import os

In [ ]:
# Setting the your Groq API key
CHATGROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"

from google.colab import userdata
CHATGROQ_API_KEY = userdata.get("GROQ_API_KEY")

In [ ]:
# Encode uploaded image to base64
def encode_image(image: Image.Image) -> str:
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

In [ ]:
# Main text function
def analyze_geography(image, prompt):
    if image is None and not prompt:
        return "Please provide at least an image or a prompt."

    messages = []
    if prompt:
        messages.append({"type": "text", "text": prompt})

    if image:
        image_b64 = encode_image(image)
        messages.append({
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_b64}"}
        })
# for the payload
    payload = {
        "model": "meta-llama/llama-4-scout-17b-16e-instruct",
        "messages": [{"role": "user", "content": messages}],
    }

 # setting headers - authenticates the request.
    headers = {
        "Authorization": f"Bearer {CHATGROQ_API_KEY}",
        "Content-Type": "application/json"
    }
 # Send the image and prompt to Groq’s API endpoint.
    response = requests.post(CHATGROQ_API_URL, json=payload, headers=headers)
    if response.status_code != 200:
        return f"Error: {response.status_code} - {response.text}"
    result = response.json()
    response_text = result["choices"][0]["message"]["content"]

    # Clean formatting: remove stars, replace line breaks with full stops
    cleaned_text = response_text.replace("*", "").replace("\n", ". ").replace("..", ".")

    return cleaned_text

In [ ]:
# Gradio UI
demo = gr.Interface(
    fn=analyze_geography,
    inputs=[
        gr.Image(type="pil", label="Upload a location image (e.g., landmark, map, street view)"),
        gr.Textbox(label="Ask about a country, region, or place")
    ],
    outputs=gr.Textbox(lines=10, label="Geographic Insight"),
    title="Geographic Analyzer",
    description="Upload a location image or ask a question about a country, region, or place. The model will identify the location and provide geographic context."
)
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://589604b8e5d6cfa481.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
